In [ ]:
import sys
import numpy as np
import os
import matplotlib.pyplot as plt

import mcg
import mdtraj as md
import time
import pandas as pd

In [ ]:
idx = 1

In [ ]:
stride = 1

In [ ]:
burst = 1e5

In [ ]:
mu = 170
run = 3

In [ ]:
folder = f"data_mu{mu}/run_{run}/"

conffolder = "gro_files"

In [ ]:
df = pd.DataFrame()

logfile_path = os.path.join(folder, 'log.txt')
weights = np.load(os.path.join(folder, 'trajectory_weights.npy'))[1:]

# Load the log file
df = pd.read_fwf(
    logfile_path,
    header=0,
    widths=[21] * 7,
    strip=True
)
df.columns = df.columns.str.strip()



In [ ]:
# df_succ = df[df['Successful']]

In [ ]:
# sp_frames = df_succ['SP frame on new traj']

In [ ]:
# sp_frames = np.array(sp_frames)

In [ ]:
# sp_frame = sp_frames[idx-1]

In [ ]:
# sp_frame

In [ ]:
# Input files
# folder = "../../data/T_250_P_500/simulation_co2_system_N_2000k_222000k_T_250_P_500_v3"
# xtc_file = os.path.join(folder, "output.xtc")
dcd_file = os.path.join(folder, f"traj_{idx}.dcd")
topology_file = os.path.join(conffolder, "conf.gro")


# Load the trajectory with a stride of n (e.g., every 10th frame)
n = stride  # Change this to the desired interval
traj = md.load(dcd_file, top=topology_file, stride=n)
original_frame_indices = np.arange(traj.n_frames) * n

traj.time = np.arange(0, traj.n_frames) * burst * n


#traj.xyz = traj.xyz.astype('float32')


# Save the reduced trajectory as a .gro file
# Output file
# gro_file = "trajectory.gro"
# traj.save_gro(gro_file)

print(f"Trajectory loaded using every {n}th step")


In [ ]:
traj = traj[:30]

In [ ]:
#traj = traj[:300]

In [ ]:
comparison_file = os.path.join(folder, f"cv_{idx}.txt")

frames_comp, mcg_values_comparison = np.loadtxt(comparison_file, skiprows=1, unpack=True)

In [ ]:
frames2us =  2e-9

In [ ]:
times_ps, CAR_COM_frames, WATER_COM_frames, box_lengths = mcg.COM_calculation_mdtraj(traj)
total_frames = len(times_ps)

MCG_OP_new_results = []

time0 = time.time()
for idx, time_ps in enumerate(times_ps):
    CAR_COM = CAR_COM_frames[idx]
    WATER_COM = WATER_COM_frames[idx]
    box_length = box_lengths[idx]
    
    # Assuming `MCG` takes these arguments and computes the order parameter
    MCG_OP, _, monomer_coords = mcg.MCG_optimized(CAR_COM, WATER_COM, box_length, guest_cutoff=0.9)
    
    #np.savetxt( '../mcg_tests/monomers.xyz', monomer_coords)
    print(f'\rProcessing frame {idx + 1}/{total_frames}', end='', flush=True)
    #print(f'The largest cluster size in the system at frame {frame} and time {round(time_ps, 10)} is: {MCG_OP}')
    MCG_OP_new_results.append(MCG_OP)
time_torch = time.time() - time0

In [ ]:
#print(mcg_values_comparison[int(sp_frame//1e5)])

In [ ]:
plt.plot(frames_comp, mcg_values_comparison, label='during TPS')
plt.plot(traj.time + 0, MCG_OP_new_results, color = 'r', zorder = 1000, label = 'from trajectory')
# plt.scatter(sxp_frame, mcg_values_comparison[int(sp_frame//1e5)], c = 'black', label = 'SP', s = 10, zorder = 10000)
plt.axhline(10, c='black')
plt.axhline(300, c='black', label='stable states')
plt.legend()
plt.ylabel('MCG')
plt.xlabel('frames')
plt.savefig('error3.png')
plt.xlim(0e8,0.02e8)
plt.ylim(0, 15)

USING THE EXACT SAME (COM) CALCULATION AS DURING TPS

In [ ]:
testlogfile_path = os.path.join(folder, 'log_test.txt')

In [ ]:
MCG_per_frame = []

co2_coms_pf = []
water_coms_pf = []
box_lengths_pf = []
 
for idx in range(traj.n_frames):
    x_p = traj.xyz[idx]
    L = traj.unitcell_vectors[idx][0][0]
    co2_coms, water_coms, box_length = mcg.CO2ClathrateCOM(x_p, L)
    box_lengths_pf.append(box_length)
    co2_coms_pf.append(co2_coms)
    water_coms_pf.append(water_coms)
    MCG_OP, _, _ = mcg.MCG_optimized(co2_coms, water_coms, box_length)
    MCG_per_frame.append(MCG_OP)

In [ ]:
plt.plot(frames_comp, mcg_values_comparison, label='during TPS')
plt.scatter(traj.time + 0, MCG_per_frame, color = 'r', zorder = 1000, label = 'per frame')
plt.plot(traj.time + 0, MCG_OP_new_results, color = 'r', zorder = 1000, label = 'from trajectory')
#plt.scatter(traj.time + 94800000, MCG_per_frame, color = 'g', zorder = 1000, label = 'from trajectory', s= 0.1)
# plt.scatter(sxp_frame, mcg_values_comparison[int(sp_frame//1e5)], c = 'black', label = 'SP', s = 10, zorder = 10000)
plt.axhline(10, c='black')
plt.axhline(300, c='black', label='stable states')
plt.legend()
plt.ylabel('MCG')
plt.xlabel('frames')
plt.savefig('error3.png')
#plt.xlim(1.75e8,1.79e8)
#plt.ylim(250, 300)
plt.xlim(0e8,0.02e8)
plt.ylim(0, 15)

In [ ]:
CAR_COM_frames = np.array(CAR_COM_frames)
co2_coms_pf = np.array(co2_coms_pf)

In [ ]:
WATER_COM_frames = np.array(WATER_COM_frames)
water_coms_pf = np.array(water_coms_pf)

In [ ]:
print(np.shape(CAR_COM_frames))

print(np.shape(co2_coms_pf))

In [ ]:
print(np.shape(CAR_COM_frames))

In [ ]:
#for i in range(10):
i = 5
plt.plot(CAR_COM_frames[:, i, 0], label = 'from traj')
plt.plot(co2_coms_pf[:, i, 0], alpha = 0.5, label='per frame')
#plt.plot(CAR_COM_frames[:, i, 0] - co2_coms_pf[:, i, 0])
plt.legend()
plt.show()
print(np.shape(traj.xyz))
#traj.xyz[:,i,0]

In [ ]:
traj_atom_5 = traj.atom_slice([16])

In [ ]:
print(traj_atom_5.xyz[:, 0])